In [ ]:
%matplotlib inline

from collections import defaultdict
from IPython import display
from PIL import Image
from torch import nn
from torch.autograd import Variable
from torchvision import models, transforms

import json
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import torch
import math
import h5py
from enum import Enum

use_cuda = 1#torch.cuda.is_available()
use_pca = 1
separate_train_videos_to_activity_regions = 1
separate_val_videos_to_activity_regions = 1
monotonic_loss_param = 60
val_id_num = 100
train_id_num = 1000 #len(train_ids)
n_epochs_for_second_training_phase = 2

In [ ]:
def one_hot(i, l):
    a = np.zeros(l, 'uint8')
    a[i] = 1
    return a

In [ ]:
data = json.load(open('data/activity_net.v1-3.min.json'))
features_file = h5py.File('data/sub_activitynet_v1-3.c3d.hdf5')

In [ ]:
#print(list(data['database'].keys()))
video_id_to_idx = {entry:i for i,entry in enumerate(list(data['database'].keys()))}
video_idx_to_id = {i:entry for i,entry in enumerate(list(data['database'].keys()))}
video_ids = list(data['database'].keys())
train_ids = []
test_ids = []
val_ids = []
for video_id in video_ids:
    if (data['database'][video_id]['subset'] == 'validation'):
        val_ids.append(video_id)
    if (data['database'][video_id]['subset'] == 'training'):
        train_ids.append(video_id)
    if (data['database'][video_id]['subset'] == 'testing'):
        test_ids.append(video_id)    
labels = set()        
for video_id in train_ids:
    labels.add(data['database'][video_id]['annotations'][0]['label'])
label_to_id = {entry:(i+1) for i,entry in enumerate(list(labels))}
label_to_id['none'] = 0
label_num = len(label_to_id)
id_to_label = {(i+1):entry for i,entry in enumerate(list(labels))}
id_to_label[0] = 'none'
#print(label_to_id)
#print(len(train_ids), len(test_ids), len(val_ids))

In [ ]:
train_id_to_categories = defaultdict(list)
train_id_to_features = defaultdict(list)
skipped = 0
new_train_ids = []
max_frame_num = 0
remove_ids = []

for video_id in train_ids: #2d array of features for each video. dim0 depends on video len. dim2=500
    
    all_features = features_file['v_' + video_id]['c3d_features']
    duration = data['database'][video_id]['duration']
    frame_num = features_file['v_' + video_id]['c3d_features'].shape[0]
    
    if (separate_train_videos_to_activity_regions==1):
        video_id_used = 0
        for i,item in enumerate(data['database'][video_id]['annotations']):
            start_frame_idx = int((item['segment'][0]/duration)*frame_num)
            end_frame_idx = min(int((item['segment'][1]/duration)*frame_num), frame_num)
            f_frame_num = end_frame_idx - start_frame_idx
            if (f_frame_num==0):
                break
            
            if (f_frame_num > max_frame_num):
                max_frame_num = f_frame_num
                
            if (video_id_used==0):
                new_video_id = video_id
                video_id_used = 1
            else: 
                new_video_id = video_id + '%d' % i;
                new_train_ids.append(new_video_id)
                
            if (start_frame_idx==0):
                train_id_to_features[new_video_id] = all_features[start_frame_idx:end_frame_idx, :]
                train_id_to_categories[new_video_id] = np.full(f_frame_num, label_to_id[item['label']]) 
            else:                
                train_id_to_features[new_video_id] = all_features[start_frame_idx-1:end_frame_idx, :]        
                train_id_to_categories[new_video_id] = np.full(f_frame_num+1, label_to_id[item['label']]) 
                train_id_to_categories[new_video_id][0] = label_to_id['none']
                
    else:
        
        train_id_to_features[video_id] = all_features       
        train_id_to_categories[video_id] = [label_to_id['none']] * frame_num
        #print(video_id, frame_num)
        for item in data['database'][video_id]['annotations']:
            start_frame_idx = ((item['segment'][0]/duration)*frame_num)
            end_frame_idx = ((item['segment'][1]/duration)*frame_num)
            #print(item['label'], start_frame_idx, end_frame_idx)
            for frame in range(int(start_frame_idx), min(frame_num, int(end_frame_idx))):
                #y = one_hot(label_to_id[item['label']], label_num)
                y = label_to_id[item['label']]
                train_id_to_categories[video_id][frame] = y
                
train_ids = train_ids + new_train_ids
#train_ids = set(train_ids)

In [ ]:
remove_ids = []
for video_id in train_ids:
    if (len(train_id_to_categories[video_id])==0):
        remove_ids.append(video_id)

for id in remove_ids:
    train_ids.remove(id)

print(len(remove_ids), len(train_ids))

In [ ]:
#print(len(train_id_to_features['6uhLrPgbpUA']), train_id_to_features['6uhLrPgbpUA'][2].shape)
#print(skipped)

print(remove_ids)
print(train_id_to_features['BrnUW2LSJDI'])

In [ ]:
val_id_to_categories = defaultdict(list)
val_id_to_features = defaultdict(list)
val_id_to_activity_regions = defaultdict(list)
skipped = 0
new_val_ids = []
remove_ids = []
for video_id in val_ids: #2d array of features for each video. dim0 depends on video len. dim2=500
    
    all_features = features_file['v_' + video_id]['c3d_features']
    duration = data['database'][video_id]['duration']
    frame_num = features_file['v_' + video_id]['c3d_features'].shape[0]
    if (separate_val_videos_to_activity_regions==1):
        video_id_used = 0
        for i,item in enumerate(data['database'][video_id]['annotations']):
            
            start_frame_idx = int((item['segment'][0]/duration)*frame_num)
            end_frame_idx = min(int((item['segment'][1]/duration)*frame_num), frame_num)
            f_frame_num = end_frame_idx - start_frame_idx
            
            if (f_frame_num<4):
                break
                
            if (video_id_used==0):
                new_video_id = video_id
                video_id_used = 1
            else: 
                new_video_id = video_id + '%d' % i;
                new_val_ids.append(new_video_id)    
                
            if (start_frame_idx==0):
                val_id_to_features[new_video_id] = all_features[start_frame_idx:end_frame_idx, :]
                val_id_to_categories[new_video_id] = np.full(f_frame_num, label_to_id[item['label']]) 
            else:                
                val_id_to_features[new_video_id] = all_features[start_frame_idx-1:end_frame_idx, :]        
                val_id_to_categories[new_video_id] = np.full(f_frame_num+1, label_to_id[item['label']]) 
                val_id_to_categories[new_video_id][0] = label_to_id['none']
        
               
    else:
        
        val_id_to_features[video_id] = all_features
        val_id_to_categories[video_id] = [label_to_id['none']] * frame_num
        #print(video_id, frame_num)
        for item in data['database'][video_id]['annotations']:
            start_frame_idx = ((item['segment'][0]/duration)*frame_num)
            end_frame_idx = ((item['segment'][1]/duration)*frame_num)
            val_id_to_activity_regions[video_id].append([int(start_frame_idx), min(frame_num, int(end_frame_idx))])
            #print(item['label'], start_frame_idx, end_frame_idx)
            for frame in range(int(start_frame_idx), min(frame_num, int(end_frame_idx))):
                val_id_to_categories[video_id][frame] = label_to_id[item['label']]
                
val_ids = val_ids + new_val_ids
val_ids = list(set(val_ids))
    

In [ ]:
remove_ids = []
for video_id in val_ids:
    if (len(val_id_to_categories[video_id])==0):
        remove_ids.append(video_id)

for id in remove_ids:
    val_ids.remove(id)

In [ ]:
#inputs - train_id_to_features
#print(len(train_id_to_features), train_id_to_features['sJFgo9H6zNo'].shape)
#outputs - train_id_to_categories
#print(len(train_id_to_categories), len(train_id_to_categories['sJFgo9H6zNo']), train_id_to_categories['sJFgo9H6zNo'][0].shape)

#print(train_id_to_features['sJFgo9H6zNo'][0,:].squeeze().shape)
#print(train_id_to_categories['sJFgo9H6zNo'][0].squeeze().shape)
print(len(remove_ids))

In [ ]:
class Detector(nn.Module):
    def __init__(self, hidden_size, input_size, output_size, use_soft_max):
        super(Detector, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.use_soft_max = use_soft_max
        if (use_soft_max):
            self.softmax = nn.Softmax(output_size)

    def forward(self, input, hidden):
        output, hidden = self.lstm(input, hidden)
        output = self.linear(output)
        if (self.use_soft_max):
            output = self.softmax(output)
        return output, hidden

    def initHidden(self, batch_size):
        result = Variable(torch.zeros(1, batch_size, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

In [ ]:
input_size = 500 if use_pca else 4096
hidden_size = 500 if use_pca else 512
output_size = label_num
learning_rate = 0.01 #consider starting from 0.01 and divide by 10 every N iterations
back_prop_depth = 1000
criterion = nn.CrossEntropyLoss()
softmax = nn.Softmax()

In [ ]:
def ResetModel():
    detector = Detector(hidden_size, input_size, output_size, 0).cuda() if use_cuda else Detector(hidden_size, input_size, output_size, 0)
    detector_optimizer = torch.optim.Adam(detector.parameters(), learning_rate)
    return detector, detector_optimizer

decoder, decoder_optimizier = ResetModel()

In [ ]:
class LossFuncType(Enum):
    CE = 1
    EARLY = 2   
    MONOTONIC = 3
    EARLY_MONOTONIC = 4
    MIXED = 5

In [ ]:
def ConvertDictTo2DArray(d):
    lend = len(d.keys())
    keys = np.fromiter(d.keys(), dtype=float)
    vals = np.fromiter(d.values(), dtype=float)
    keys.shape = (lend, 1)
    vals.shape = (lend, 1)
    return tuple(map(tuple, np.append(keys, vals, axis=1).astype(int)))

In [ ]:
class CrossEntropyLossFunc(torch.nn.Module):
    
    def __init__(self):
        super(CrossEntropyLossFunc, self).__init__()
        
    def forward(self,detector_output, target_variable, target_variable_one_hot):
        criterion = nn.CrossEntropyLoss()
        loss = 0
        loss_arr_scalar = []
        for di in range(target_variable.shape[0]):
            curr_target = target_variable[di] #1x201
            #curr_target = curr_target.cuda() if use_cuda else curr_target         
            loss_step = criterion(detector_output[di], curr_target)
            loss += loss_step
            loss_arr_scalar.append(loss_step.cpu().data[0])
                
        return loss, loss_arr_scalar
    
class EarlyLossFunc(torch.nn.Module):
    
    def __init__(self):
        super(EarlyLossFunc,self).__init__()
        
    def forward(self,detector_output, target_variable, target_variable_one_hot):
        loss = 0
        T = target_variable.shape[0]
        N = label_num
        B = target_variable.shape[1]
        #print(B)
        loss_arr_scalar = []
        detector_output_after_softmax = softmax(detector_output)
        detector_output_after_softmax_one_hot = detector_output_after_softmax * target_variable_one_hot
        detector_output_after_softmax_one_hot_inverse = 1 - detector_output_after_softmax * target_variable_one_hot

        for di in range(T): #di=0..T-1

            false_neg = (-((di+1)/(T+1)) * detector_output_after_softmax_one_hot_inverse[di, :, :] * 
                                torch.log1p(-detector_output_after_softmax[di, :, :]))

            false_pos = (-detector_output_after_softmax_one_hot[di, :, :] * 
                         torch.log(detector_output_after_softmax[di, :, :]))
 
            loss_step = (false_neg.sum() + false_pos.sum())
            
            loss += loss_step
            loss_arr_scalar.append(loss_step.cpu().data[0])
    
        return loss, loss_arr_scalar

    
class MonotonicLossFunc(torch.nn.Module):
    
    def __init__(self, param):
        super(MonotonicLossFunc, self).__init__()
        self.param = param
        
    def forward(self,detector_output, target_variable, target_variable_one_hot):
        loss = 0
        T = target_variable.shape[0]
        N = label_num
        B = target_variable.shape[1]
        criterion = nn.CrossEntropyLoss()
        loss_arr_scalar = []
        detector_output_after_softmax = softmax(detector_output)
        detector_output_after_softmax_one_hot = detector_output_after_softmax * target_variable_one_hot
        
        for di in range(T): #di=0..T-1
     
            if (di==0):
                loss_r = 0
            else:
                current_activity_list = detector_output_after_softmax_one_hot[0:di, :, :]
                max_last_score, indices = torch.max(current_activity_list, dim=0)
                #print("max_score: ", max_last_score.data, "curr score: ", detector_output_after_softmax_one_hot[di, :, :].data)
                loss_r_matrix = torch.clamp(max_last_score - detector_output_after_softmax_one_hot[di, :, :], min=0)
                #print("loss matrix: ", loss_r_matrix, loss_r_matrix.sum())
                loss_r = loss_r_matrix.sum()/B       
            
            loss_c = criterion(detector_output[di], target_variable[di])  
            if (di>0):
                print(loss_c.data[0], loss_r.data[0])
            else:
                print(loss_c.data[0], loss_r)
                
            loss_step = loss_c + self.param * loss_r
            loss += loss_step
            loss_arr_scalar.append(loss_step.cpu().data[0])
            
        
        return loss, loss_arr_scalar
    
class EarlyMonotonicLossFunc(torch.nn.Module):
    
    def __init__(self, param):
        super(EarlyMonotonicLossFunc, self).__init__()
        self.param = param
        
    def forward(self,detector_output, target_variable, target_variable_one_hot):
        loss = 0
        T = target_variable.shape[0]
        N = label_num
        B = target_variable.shape[1]
        criterion = nn.CrossEntropyLoss()
        loss_arr_scalar = []
        detector_output_after_softmax = softmax(detector_output)
        detector_output_after_softmax_one_hot = detector_output_after_softmax * target_variable_one_hot
        
        for di in range(T): #di=0..T-1
     
            if (di==0):
                loss_r = 0
            else:
                current_activity_list = detector_output_after_softmax_one_hot[0:di, :, :]
                max_last_score, indices = torch.max(current_activity_list, dim=0)
                loss_r_matrix = torch.clamp(max_last_score - detector_output_after_softmax_one_hot[di, :, :], min=0)
                loss_r = loss_r_matrix.sum()/B       
            
            loss_c = criterion(detector_output[di], target_variable[di])            
            loss_step = ((di+1)/(T+1)) * ( loss_c + self.param * loss_r )
            loss += loss_step
            loss_arr_scalar.append(loss_step.cpu().data[0])
        
        return loss, loss_arr_scalar   

In [ ]:
def train_iteration(target_variable, target_variable_one_hot, input_variable, 
          model, 
          optimizer, 
          loss_func, frame_num): 
    
    optimizer.zero_grad()
    target_length = frame_num #len(target_variable)
    batch_size = target_variable.shape[1]
    #print(batch_size, target_variable.shape)
    loss_arr_per_frame = []
    detector_hidden = detector.initHidden(batch_size)
    detector_hidden = detector_hidden.cuda() if use_cuda else detector_hidden
    detector_cells = detector.initHidden(batch_size)
    detector_cells = detector_cells.cuda() if use_cuda else detector_cells
    
    detector_output = []
    detector_output = Variable(torch.zeros(target_length, batch_size, label_num))
    detector_output = detector_output.cuda() if use_cuda else detector_output
    #print(len(target_variable))
    
    score_per_frame = []
    
    for di in range(target_length):
        #input len 500
        detector_input = np.array(input_variable[di, :, :]) #1x500
        detector_input = Variable(torch.FloatTensor(detector_input)).view(1,batch_size,-1)
        detector_input = detector_input.cuda() if use_cuda else detector_input
        #print(detector_input.shape, detector_hidden.shape)
        detector_output_single, (detector_hidden, detector_cells) = model(detector_input, (detector_hidden, detector_cells))
        detector_output[di] = detector_output_single
        #topv, topi= softmax(detector_output_single).data.topk(1)
        #ind_pos = target_variable[di].astype(int).tolist()
        #print((detector_output_single).shape, (detector_input).shape)
        curr_cat = target_variable[di, 0].cpu().data[0];
        topv, topi = softmax(detector_output_single[0, :, :].squeeze()).data.topk(1)
        #print("top score = ", topv[0])
        #score_per_frame.append(softmax(detector_output_single[0, 0, :])[curr_cat].data[0])
        #print(topv.shape)
        score_per_frame.append(topv.sum()/batch_size)
     
    loss, loss_arr_scalar = loss_func(detector_output, target_variable, target_variable_one_hot)    
    if (back_prop_depth > target_length):
        loss.backward()
        #print(loss)
        optimizer.step()
        
        
    return loss.cpu().data[0], score_per_frame, loss_arr_scalar

In [ ]:
learning_rate = 0.01
def FindLongestVideoLenInABatch(ids):
    result = 0
    for id in ids:
        if (len(train_id_to_categories[id]) == 0):
            print("ERROR: FindLongestVideoLenInABatch id= ", id)
        if (train_id_to_categories[id].shape[0] > result):
            
            result = train_id_to_categories[id].shape[0]
    return result

def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 10 epochs"""
    lr = learning_rate * (0.1 ** (epoch // 5))
    print("set lr to ", lr, " on epoch ", epoch)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
def train(nepochs, batch_size, train_id_num, model, optimizer, loss_func, loss_type, second_loss_func = EarlyLossFunc()):
    loss_arr_per_frame = []
    loss_progress = []
    iter_print_freq = 20
    nbatches = train_id_num/batch_size
    print("Staring training. nepochs=", nepochs, " nbatches=", nbatches, " batch_size=", batch_size)
    for iter in range(nepochs):
        adjust_learning_rate(optimizer, iter)
        for bid in range(int(nbatches)):
            max_frame_num = FindLongestVideoLenInABatch(train_ids[bid*batch_size : (bid+1)*batch_size])
            #print("Longest video len = ", max_frame_num)
            target_variable = np.zeros((max_frame_num, batch_size), dtype=int)
            target_variable_one_hot = np.zeros((max_frame_num, batch_size, label_num), dtype=int)
            input_variable = np.zeros((max_frame_num, batch_size, input_size))
            
            for i,train_id in enumerate(train_ids[bid*batch_size : (bid+1)*batch_size]):
                curr_frame_num = train_id_to_categories[train_id].shape[0]
                target_variable[0:curr_frame_num, i] = train_id_to_categories[train_id]
                target_variable_one_hot[0:curr_frame_num, i, :] = one_hot(train_id_to_categories[train_id], label_num)
                input_variable[0:curr_frame_num, i, :] = train_id_to_features[train_id];
               
            if (loss_type == LossFuncType.MIXED and (iter>=(nepochs-n_epochs_for_second_training_phase))):
                loss_func = second_loss_func
                #print("Starting second training phase for MIXED loss function")
            
            target_variable = Variable(torch.LongTensor(target_variable))
            target_variable = target_variable.cuda() if use_cuda else target_variable
            
            target_variable_one_hot = Variable(torch.FloatTensor(target_variable_one_hot))
            target_variable_one_hot = target_variable_one_hot.cuda() if use_cuda else target_variable_one_hot
            
            loss, score_per_frame, loss_arr_scalar = train_iteration(target_variable, target_variable_one_hot, input_variable, model, optimizer, loss_func, max_frame_num)
            np.resize(score_per_frame, (1, train_id_to_categories[train_ids[bid*batch_size]].shape[0]))
            curr_average_loss = loss/max_frame_num;
            if ((bid % iter_print_freq) == 0):
                loss_progress.append(curr_average_loss)
                print("epoch: ", iter, "batch: ", bid, "loss: ", curr_average_loss)
                
            if (bid >= train_id_num):
                break
          
    return loss_arr_scalar, score_per_frame

In [ ]:
def GetLossFunc(loss_type):
    if (loss_type == LossFuncType.CE):
        return CrossEntropyLossFunc()
    if (loss_type == LossFuncType.EARLY):
        return EarlyLossFunc()
    if (loss_type == LossFuncType.MONOTONIC):
        return MonotonicLossFunc(monotonic_loss_param)
    if (loss_type == LossFuncType.EARLY_MONOTONIC):
        return EarlyMonotonicLossFunc(monotonic_loss_param) 
    if (loss_type == LossFuncType.MIXED):
        return CrossEntropyLossFunc()
    
def GetSecondMixedLossFunc():
    return EarlyMonotonicLossFunc(monotonic_loss_param) 

def GetModelName(loss_type):
    if (loss_type == LossFuncType.CE):
        return 'detector_ce.pt'
    if (loss_type == LossFuncType.EARLY):
        return 'detector_early.pt'
    if (loss_type == LossFuncType.MONOTONIC):
        return 'detector_monotonic.pt'
    if (loss_type == LossFuncType.EARLY_MONOTONIC):
        return 'detector_early_monotonic.pt'
    if (loss_type == LossFuncType.MIXED):
        return 'detector_mixed.pt'

In [ ]:
nepochs = 20
batch_size = 64
train_id_num = 6400
learning_rate = 0.01
#loss_types = [LossFuncType.CE, LossFuncType.MONOTONIC, LossFuncType.EARLY, LossFuncType.EARLY_MONOTONIC, LossFuncType.MIXED]
loss_types = [LossFuncType.CE, LossFuncType.MONOTONIC, LossFuncType.EARLY_MONOTONIC]
loss_arrs_per_frame = []
scores_per_frame = []
models = []
for loss_type in loss_types:
    loss_func = GetLossFunc(loss_type)
    detector, detector_optimizer = ResetModel()
    loss_arr_per_frame, score_per_frame = train(nepochs, batch_size, train_id_num, detector, detector_optimizer, loss_func, loss_type, second_loss_func = GetSecondMixedLossFunc())
    loss_arrs_per_frame.append(loss_arr_per_frame)
    scores_per_frame.append(score_per_frame)
    model_name = GetModelName(loss_type)    
    torch.save(detector, model_name)
    models.append(detector.cpu())
    

In [ ]:
np.save('scores_per_frame.npy', scores_per_frame)
np.save('loss_arrs_per_frame.npy', loss_arrs_per_frame)

In [ ]:
scores_per_frame = np.load('scores_per_frame.npy')
loss_arrs_per_frame = np.load('loss_arrs_per_frame.npy')

plt.figure(1)
plt.title('scores per frame')
for i,loss_type in enumerate(loss_types):
    plt.plot(scores_per_frame[i][0:40], label=GetModelName(loss_type))
    
plt.legend()
plt.show()

plt.figure(2)
plt.title('loss_arrs_per_frame')
for i,loss_type in enumerate(loss_types):
    plt.plot(loss_arrs_per_frame[i][0:40], label=GetModelName(loss_type))
    
plt.legend()
plt.show()

In [ ]:
def inference_iteration(detector, input_variable):
    detector_hidden = detector.initHidden(1)
    detector_hidden = detector_hidden.cuda() if use_cuda else detector_hidden
    
    detector_cells = detector.initHidden(1)
    detector_cells = detector_cells.cuda() if use_cuda else detector_cells
    
    input_len = input_variable.shape[0]
    categories_per_frame = []
    scores_per_frame = []
    
    for di in range(input_len):
        #input len 500
        detector_input = np.array(input_variable[di,:]) #1x500
        detector_input = Variable(torch.FloatTensor(detector_input)).view(1,1,-1)
        detector_input = detector_input.cuda() if use_cuda else detector_input
        #
        detector_output, (detector_hidden, detector_cells) = detector(detector_input, (detector_hidden, detector_cells))
        #topv, topi= softmax(detector_output).data.topk(5)
        
        topv, topi = softmax(detector_output[0, 0, :]).data.topk(1)
        #print("top score = ", topv[0])
        #score_per_frame.append(topv[0])
        ni = topi[0] #category id
        nv = topv[0] #score
        categories_per_frame.append(ni)
        scores_per_frame.append(nv)
            
    return np.array(categories_per_frame), np.array(scores_per_frame) #nframesx1 each

In [ ]:
def inference(model_name, val_id_num):
    print("inference called with model_name: ", model_name)
    all_categories = []
    all_scores = []
    all_categories_true = []
    model = torch.load(model_name)
    model.lstm.flatten_parameters()
    max_val_video_len = 0
    binary_success = []
    for i,val_id in enumerate(val_ids):
        input_variable = val_id_to_features[val_id];
        categories, scores = inference_iteration(model, input_variable)
        if (max_val_video_len < input_variable.shape[0]):
            max_val_video_len = input_variable.shape[0]
        all_categories.append(categories)
        all_scores.append(scores)
        all_categories_true.append(val_id_to_categories[val_id])
        binary_success.append((categories == np.array(val_id_to_categories[val_id])).astype(int))
        if (i >= val_id_num):
            break

    return binary_success

In [ ]:
def CropAndPad(input_res, crop_frame):
    for i in range(len(input_res)):
        #remove the first one, since its non activity
        #input_res[i] = input_res[i][1:input_res[i].shape[0]];
    
        if (input_res[i].shape[0] > crop_frame):
            input_res[i] = input_res[i][0:crop_frame]
        else:
            pad_size = crop_frame - input_res[i].shape[0]
            input_res[i] = np.pad(input_res[i], (0, pad_size), 'edge')
        if (input_res[i].shape[0] != crop_frame):
            print(input_res[i].shape[0])
        
    return input_res

In [ ]:
#loss_types = [LossFuncType.MONOTONIC]
loss_types = [LossFuncType.CE, LossFuncType.MONOTONIC, LossFuncType.EARLY_MONOTONIC]
average_accuracy_per_frame = []
val_id_num = 100
for i,loss_type in enumerate(loss_types):
    binary_classify_success = inference(GetModelName(loss_type), val_id_num)
    binary_classify_success = CropAndPad(binary_classify_success, crop_frame=20)
    average_accuracy_per_frame.append(sum(binary_classify_success)/len(binary_classify_success))

In [ ]:
plt.figure(3)
plt.title('average_accuracy_per_frame')
for i,loss_type in enumerate(loss_types):
    plt.plot(average_accuracy_per_frame[i], label=GetModelName(loss_type))
    
plt.legend()
plt.show()